<a href="https://colab.research.google.com/github/cvcdrew/WeatherWise-Andrew/blob/main/Copy_of_starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook



In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus
!pip install customtkinter


In [ ]:
import os

os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

Enter your API key: 050d8193513b67975f932a4bb3b864de


## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
import requests
import matplotlib.pyplot as plt
import pyinputplus as pyip
import pandas as pd
from datetime import datetime, timedelta

import seaborn as sns
# ✅ Import after installing (if needed)
from fetch_my_weather import get_weather
from hands_on_ai.chat import get_response

# Add any other setup code here

## 🌤️ Weather Data Functions

In [ ]:
import requests
from typing import Dict

API_KEY = "your_api_key_here"
BASE_URL = "https://api.openweathermap.org/data/2.5/forecast"
UNITS = "metric"
INTERVALS_PER_DAY = 8  # 3-hour intervals

def build_weather_query_params(location: str, forecast_days: int) -> Dict[str, str]:
    """Build query parameters for the weather API request."""
    return {
        "q": location,
        "cnt": forecast_days * INTERVALS_PER_DAY,
        "units": UNITS,
        "appid": API_KEY
    }

def fetch_weather_data(params: dict) -> dict:
    """Fetch weather data from the API."""
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"[ERROR] Failed to fetch weather data: {e}")
        return {}

def get_weather_data(location: str, forecast_days: int = 5) -> dict:

    params = build_weather_query_params(location, forecast_days)
    return fetch_weather_data(params)


## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display', max_points=200):
    df = preprocess_weather_data(weather_data['date'], weather_data['temperature'], max_points)

    fig, ax = plt.subplots(figsize=(10, 5))
    sns.lineplot(data=df, x='Date', y='Value', marker='o', ax=ax)
    ax.set_title('Temperature Trend')
    ax.set_xlabel('Date')
    ax.set_ylabel('Temperature (°C)')
    ax.grid(True)
    fig.autofmt_xdate()
    fig.tight_layout()

    if output_type == 'figure':
        plt.close(fig)
        return fig
    else:
        plt.show()


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def create_precipitation_visualisation(weather_data, output_type='display'):

    # Convert weather_data to DataFrame for easier plotting
    df = pd.DataFrame({
        'Date': pd.to_datetime(weather_data['date']),
        'Precipitation': weather_data['precipitation']
    })

    plt.figure(figsize=(10, 5))
    sns.barplot(data=df, x='Date', y='Precipitation', color='skyblue')
    plt.title('Precipitation Forecast')
    plt.xlabel('Date')
    plt.ylabel('Precipitation (mm)')
    plt.xticks(rotation=45)
    plt.tight_layout()

    if output_type == 'figure':
        fig = plt.gcf()  # Get current figure
        plt.close(fig)   # Prevent display if returning
        return fig
    else:
        plt.show()


## 🤖 Natural Language Processing

In [ ]:
import re

# Configurable rules
TIME_PERIOD_PATTERNS = {
    'today': [r'\btoday\b'],
    'tomorrow': [r'\btomorrow\b'],
    'week': [r'\bnext\s?7\s?days\b', r'\b7\s?days\b', r'\bweek\b']
}

WEATHER_ATTRIBUTES = [
    'temperature', 'rain', 'precipitation', 'humidity',
    'wind', 'sunny', 'cloudy', 'forecast'
]


def extract_location(question: str) -> str:
    """Extract location from the question."""
    match = re.search(r'\bin ([a-zA-Z\s]+)', question)
    if match:
        return match.group(1).strip()

    # Fallback: last word that isn't a known keyword
    tokens = question.split()
    for word in reversed(tokens):
        if word not in WEATHER_ATTRIBUTES and not any(tp in word for patterns in TIME_PERIOD_PATTERNS.values() for tp in patterns):
            return word
    return None


def extract_time_period(question: str) -> str:
    """Extract time period from the question."""
    for label, patterns in TIME_PERIOD_PATTERNS.items():
        for pattern in patterns:
            if re.search(pattern, question):
                return label
    return 'today'  # Default


def extract_weather_attribute(question: str) -> str:
    """Extract the weather attribute from the question."""
    for attr in WEATHER_ATTRIBUTES:
        if attr in question:
            return attr
    return None


def parse_weather_question(question: str) -> dict:

    q = question.lower()

    return {
        'location': extract_location(q),
        'time_period': extract_time_period(q),
        'weather_attribute': extract_weather_attribute(q)
    }


## 🧭 User Interface

In [ ]:
# Define menu functions using pyinputplus or ipywidgets here

import customtkinter as ctk
from tkinter import messagebox

ctk.set_appearance_mode("System")  # Light or Dark
ctk.set_default_color_theme("blue")  # Or "dark-blue", "green"

def run_app():
    def on_submit():
        loc = location_entry.get()
        days = int(days_entry.get())
        weather_data = get_weather_data(loc, days)
        if visual_type.get() == "Temperature":
            create_temperature_visualisation(weather_data)
        else:
            create_precipitation_visualisation(weather_data)

    app = ctk.CTk()
    app.geometry("400x300")
    app.title("WeatherWise")

    ctk.CTkLabel(app, text="WeatherWise", font=("Arial", 20)).pack(pady=10)

    location_entry = ctk.CTkEntry(app, placeholder_text="Enter location")
    location_entry.pack(pady=10)

    days_entry = ctk.CTkEntry(app, placeholder_text="Forecast days (1-7)")
    days_entry.pack(pady=10)

    visual_type = ctk.CTkOptionMenu(app, values=["Temperature", "Precipitation"])
    visual_type.set("Temperature")
    visual_type.pack(pady=10)

    submit_btn = ctk.CTkButton(app, text="Show Visualization", command=on_submit)
    submit_btn.pack(pady=20)

    app.mainloop()


ModuleNotFoundError: No module named 'customtkinter'

## 🧩 Main Application Logic

In [ ]:
# Tie everything together here
def generate_weather_response(parsed_question, weather_data):
    """
    Generate a natural language response to a weather question.

    Args:
        parsed_question (dict): Parsed question data
        weather_data (dict): Weather data

    Returns:
        str: Natural language response
    """
    pass

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.